## Learning ipywidgets: a very simple d3 "CircleWidget" with a js CircleView 


I use this notebook to get acquainted with the mechanics and best practices for coding interactive ipywidgets, for Jupyter notebooks and the forthcoming Jupyter lab.    If successful, it may be useful to others who have already worked through the excellent [tutorial and 'datepicker' examples](https://ipywidgets.readthedocs.io/en/latest/) provided by the ipywidgets team.

Once this simple notebook, illustrating the use of a "foreign" graphics library with ipywidgets, has passed muster, I will add documentation on the extra steps to produce a proper cookiecutter nbextensions template, and eventually into jupyter lab.

 Make the d3 library available via requirejs.  Repeated attempts to also use define and shims failed to produce a
d3 variable to be used for drawing.  See crude solution, with require(["d3"]) in function createDiv along with
an embarrassing use of global "window" so d3 can be referenced in createCanvasDrawCircle.

Possibly related is this 2014 [discussion](https://github.com/d3/d3/issues/1693): d3 vs. AMD, support for require but maybe not define.

In [1]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    },
});


<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as widgets
from traitlets import Int, Unicode, validate

class CircleWidget(widgets.DOMWidget):
    _view_name = Unicode('CircleView').tag(sync=True)
    _view_module = Unicode('circle').tag(sync=True)
    ratio = Int(100).tag(sync=True)


The CircleView object, below, deletes and recreates both svg canvas and a circle on every "ratio changed" event.
An obvious improvement is to allows the svg canvas, at least, to persist.  I could not make that work:  every attempt to keep that variable around as part of CircleView failed.  I also failed to get an
initial circle drawn.  Perhaps there is some DOM and/or svg latency?  

In [3]:
%%javascript
require.undef('circle');

define('circle', ["jupyter-js-widgets"], function(widgets) {
    
    var CircleView = widgets.DOMWidgetView.extend({

        createCanvasDrawCircle: function(ratio){
           window.d3.select("#svg").remove();
           var svg = window.d3.select("#d3DemoDiv")
                              .append("svg")
                              .attr("id", "svg").attr("width", 600).attr("height", 300);
           svg.append("circle")
              .style("stroke", "gray")
              .style("fill", "white")
              .attr("r", ratio)
              .attr("cx", 300)
              .attr("cy", 150)
              .on("mouseover", function(){d3.select(this).style("fill", "aliceblue");})
              .on("mouseout", function(){d3.select(this).style("fill", "white");});
            return(svg)
            },

        createDiv: function(){
           require(["d3"], function(d3){  // require, with and w/o shim, failed to load d3
              console.log(d3.version);
              window.d3 = d3;
              });
            var div = $("<div id='d3DemoDiv' style='border:1px solid red; height: 300px; width: 600px'>")
            return(div);
            },
 
        render: function() { 
            console.log("CircleView:render called, current ratio is " + this.model.get('ratio'));
            this.$el.append(this.createDiv());
            this._ratio_changed();  // 
            this.listenTo(this.model, 'change:ratio', this._ratio_changed, this);
           },

        _ratio_changed: function() {
           console.log("CircleView:_ratio_changed called")
           var newRatio = this.model.get('ratio');
           var canvas = this.createCanvasDrawCircle(newRatio);
           }
      });
    return {
        CircleView : CircleView
    };
});

<IPython.core.display.Javascript object>

Call the ctor, evaluate the object: render method called, but no drawing takes place.  dom/d3 latency?

In [7]:
cw = CircleWidget(height=300, width=500)  # render called, _ratio_changed called, but no circle drawn
cw

Modifiying the traitlet Int ratio field triggers ratio_changed, circle is drawn with new radius

In [12]:
cw.ratio = 80

In [11]:
cw.ratio = 40